# Image-wise operation

This notebook demostrates common image-wise operations by using SDK.

## Prerequisites

You need

- an SDK environment set up with the administrator priviledge. Check "*environment setup.ipynb*" for the detail.

**Replace the value below with your PrimeHub domain (*http* or *https*).**

In [ ]:
PRIMEHUB_CLUSTER = 'https://c.demo.primehub.io'

**Check if the configuration exists. If not, you need to login to retrieve the token and input it in the prompt. The configuration will be generated at**`~/.primehub/config.json`.

In [ ]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print(f"PrimeHub Python SDK {ph.version.version()} environment is ready, you are good to go.")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("Failed to retrieve the information from PrimeHub cluster, please check the configuration.")

## ToC

- [Fundamental Operations (CRUD)](#fundamental)
- [Common Operations](#common)

---
## Fundamental Operations <a class="anchor" id="fundamental"></a>

In [ ]:
# Get the group id by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        specified_group = group
        
group_detail = ph.admin.groups.get(specified_group['id'])
group_detail

### Add existing Image from Image Registry

Method: `ph.admin.images.create(image_configuration)`

You can opt for adding a *global image* or adding *an image connecting specified groups*.

In [ ]:
image_config_global = {
  "name": "sdk-test-base-global",
  "displayName": "Base image by SDK",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": True
  #"useImagePullSecret": 'secret_id'
}


image_config_with_specified_groups = {
  "name": "sdk-test-base",
  "displayName": "Base image by SDK with specified groups",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": False,
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
      }
    ]
  },
  #"useImagePullSecret": 'secret_id'
}


In [ ]:
created_image = ph.admin.images.create(image_config_with_specified_groups)
created_image

### Update Image Configuration

Method: `ph.admin.images.update('image_id', configuration)`

In [ ]:
updating_config = {
    "description": "Base image updated by SDK example"
}

ph.admin.images.update(created_image['id'], updating_config)

### Get Image Detail Info

Method: `ph.admin.images.get('image_'id)`

In [ ]:
ph.admin.images.get(created_image['id'])

### Get connected Groups of Image

Method: `ph.admin.images.list_groups('image_id')`

In [ ]:
ph.admin.images.list_groups(created_image['id'])

### List All of Images

Method: `ph.admin.images.list()`

In [ ]:
all_images = ph.admin.images.list()
for image in all_images:
    print(image)

### Delete Image

Method: `ph.admin.images.delete('image_id')`

In [ ]:
ph.admin.images.delete(created_image['id'])

---

## Common Operations <a class="anchor" id="common"></a>

### Connect/Disconnect Image to Group

Method: 
- `ph.admin.images.add_group('image_id', 'group_id')`
- `ph.admin.images.remove_group('image_id', 'group_id')`

In [ ]:
image_id = 'bitfusion-tf-cuda11'
ph.admin.images.add_group(image_id, specified_group['id'])

# Verify
ph.admin.images.list_groups(image_id)

In [ ]:
ph.admin.images.remove_group(image_id, specified_group['id'])

# Verify
ph.admin.images.list_groups(image_id)

### Create ImagePull secret

By using `ph.admin.secrets.create()` to create the pull secret.

In [ ]:
secret_config = {
    "name": "secret-created-by-sdk",
    "type": "opaque",
    "secret": "keep it secret"
}
secret = ph.admin.secrets.create(secret_config)
secret

### Add Image by Building a custom image

By using `ph.admin.images.create()` with the image configruation containing `imageSpec` of the custom image.

In [ ]:
# Create an custom image
config = {
  "name": "custom-image-built-by-sdk",
  "type": "both",
  "global": True,
  "imageSpec": {
    "baseImage": "jupyter/base-notebook",
    "packages": {
      "pip": [
        "primehub-python-sdk"
      ]
    }
  }
}

In [ ]:
image_custom = ph.admin.images.create(config)

We can tell the building status from `jobStatus` by getting the custom image info.

In [ ]:
output = ph.admin.images.get(image_custom['id'])

# "logEndpoint" will output massive logs of the building job; usually we delete it to avoid a lengthy output 
# unless you need to check.
del output['logEndpoint']

output